In [2]:
from bs4 import BeautifulSoup

In [3]:
with open("data.html", "r", encoding="utf-8") as file:
    html_content = file.read()

In [4]:
soup = BeautifulSoup(html_content, "html.parser")

In [16]:
documents = []
for element in soup.find_all(class_="panel-default"):
    name = element.find("a").text.strip()
    # if "Всі настанови. Формат для друку (PDF). Версія 1810T2P2Q1R2".strip() in name or "Аудіо. Версія 1810T0P0Q0R0".strip() in name or "Зображення. Версія 1810T0P0Q0R0".strip() in name:
    #     continue

    if not "General practice\Загальна практика".strip() in name:
        continue

    docs = element.find_all("a")
    for a_tag in docs:
        if "/documents/" in a_tag["href"]:
            title_tag = a_tag.find("span", class_="document-title")
            if title_tag:
                title = title_tag.get_text(strip=True).replace(">> ", "")
                link = f"https://guidelines.moz.gov.ua{a_tag['href']}"
                documents.append({"name": title, "link" : link})

In [ ]:
import os
import requests

In [ ]:
import os
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium_stealth import stealth

# Папка для загрузки документов
download_dir = os.path.join(os.getcwd(), "documents")
os.makedirs(download_dir, exist_ok=True)

# Настройка опций Chrome
chrome_options = Options()
# Указываем путь для автоматической загрузки файлов
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
chrome_options.add_experimental_option("prefs", prefs)

# Добавляем аргументы для имитации реального браузера
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--disable-popup-blocking")
# Указываем реалистичный User-Agent (пример для Mac)
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                            "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36")
# Рекомендуется не использовать headless-режим для обхода защиты Cloudflare
chrome_options.headless = False

# Инициализация ChromeDriver (укажите корректный путь к chromedriver, если он не в PATH)
service = Service('/usr/local/bin/chromedriver')
driver = webdriver.Chrome(service=service, options=chrome_options)

# Применяем selenium-stealth для сокрытия автоматизации
stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="MacIntel",         # для Mac
        webgl_vendor="Apple Inc.",   # типичное значение для macOS
        renderer="Apple GPU",        # значение можно уточнить под вашу систему
        fix_hairline=True)

def wait_for_new_file(before_files, folder, timeout=30):
    """
    Функция ожидает появления нового файла в папке folder,
    которого не было в before_files.
    Возвращает имя нового файла или None, если файл не появился за timeout секунд.
    """
    seconds = 0
    while seconds < timeout:
        current_files = set(os.listdir(folder))
        new_files = current_files - before_files
        # Для Chrome временные файлы обычно имеют расширение .crdownload
        new_files = [f for f in new_files if not f.endswith('.crdownload')]
        if new_files:
            return new_files[0]
        time.sleep(1)
        seconds += 1
    return None


for doc in tqdm(documents):
    url = doc['link']
    name = doc['name']

    # Сохраняем список файлов в папке загрузок до начала скачивания
    before = set(os.listdir(download_dir))

    # Переходим по ссылке документа
    driver.get(url)

    # Даем время на начало загрузки файла (настройте задержку по необходимости)
    time.sleep(5)

    # Ожидаем появления нового файла в папке загрузки
    new_file = wait_for_new_file(before, download_dir, timeout=30)
    if new_file:
        # Определяем расширение файла
        ext = os.path.splitext(new_file)[1]
        # Формируем безопасное имя файла (удаляем недопустимые символы)
        safe_name = "".join(c for c in name if c not in r'\/:*?"<>|')
        new_name = safe_name + ext
        old_path = os.path.join(download_dir, new_file)
        new_path = os.path.join(download_dir, new_name)
        os.rename(old_path, new_path)
        print(f"Скачан: {new_path}")
    else:
        print(f"Не удалось скачать файл по ссылке: {url}")

driver.quit()


In [17]:
import os
import time
from tqdm import tqdm
import undetected_chromedriver as uc

def wait_for_new_file(before_files, folder, timeout=30):
    """
    Функция ожидает появления нового файла в папке folder,
    которого не было в before_files.
    Возвращает имя нового файла или None, если файл не появился за timeout секунд.
    """
    seconds = 0
    while seconds < timeout:
        current_files = set(os.listdir(folder))
        new_files = current_files - before_files
        # Для Chrome временные файлы обычно имеют расширение .crdownload
        new_files = [f for f in new_files if not f.endswith('.crdownload')]
        if new_files:
            return new_files[0]
        time.sleep(1)
        seconds += 1
    return None

# Папка для загрузки документов
download_dir = os.path.join(os.getcwd(), "documents")
os.makedirs(download_dir, exist_ok=True)

# Настройка опций для undetected‑chromedriver
options = uc.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
    # Отключаем встроенный просмотр PDF, чтобы файлы скачивались
    "plugins.always_open_pdf_externally": True
}
options.add_experimental_option("prefs", prefs)

# Добавляем аргументы для имитации реального браузера
options.add_argument("start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                     "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36")
# Рекомендуется не использовать headless-режим для обхода защиты Cloudflare
options.headless = False

# Инициализируем undetected‑chromedriver
driver = uc.Chrome(options=options)

for doc in tqdm(documents):
    url = doc['link']
    name = doc['name']

    # Сохраняем список файлов в папке загрузок до начала скачивания
    before = set(os.listdir(download_dir))

    # Переходим по ссылке документа
    driver.get(url)

    # Ждём, пока начнётся загрузка файла (при необходимости можно увеличить задержку)
    time.sleep(5)

    # Ожидаем появления нового файла в папке загрузок
    new_file = wait_for_new_file(before, download_dir, timeout=30)
    if new_file:
        # Определяем расширение файла
        ext = os.path.splitext(new_file)[1]
        # Формируем "безопасное" имя файла из значения словаря
        safe_name = "".join(c for c in name if c not in r'\/:*?"<>|')
        new_name = safe_name + ext
        old_path = os.path.join(download_dir, new_file)
        new_path = os.path.join(download_dir, new_name)

        # Если файл с таким именем уже существует, можно его удалить (или применить другой способ обработки)
        if os.path.exists(new_path):
            os.remove(new_path)

        os.rename(old_path, new_path)
        print(f"Скачан: {new_path}")
    else:
        print(f"Не удалось скачать файл по ссылке: {url}")

driver.quit()


  4%|▎         | 1/28 [00:05<02:30,  5.56s/it]

Скачан: /Users/antonnovokhatskiy/Desktop/33/health/prepare_data/documents/003.002 ebm00787 Втома.pdf


  7%|▋         | 2/28 [00:10<02:21,  5.46s/it]

Скачан: /Users/antonnovokhatskiy/Desktop/33/health/prepare_data/documents/003.003 ebm00931 Хронічний біль.pdf


 11%|█         | 3/28 [00:16<02:15,  5.41s/it]

...

Скачан: /Users/antonnovokhatskiy/Desktop/33/health/prepare_data/documents/003.048 ebm00984 Cкелетно-м'язовий біль.pdf


 96%|█████████▋| 27/28 [02:25<00:05,  5.35s/it]

Скачан: /Users/antonnovokhatskiy/Desktop/33/health/prepare_data/documents/003.060 ebm01142 Роль мотиваційного інтерв’ю у лікуванні та модифікації способу життя.pdf


100%|██████████| 28/28 [02:30<00:00,  5.38s/it]

Скачан: /Users/antonnovokhatskiy/Desktop/33/health/prepare_data/documents/038.020 ebm00007 Фарингіт і тонзиліт.pdf
